<a href="https://colab.research.google.com/github/hardik-05/Custom_Llama2/blob/training_data/Llama2_1000_1_epoch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Fine Tune LLama 2 on hardikch05/Text_to_sql_custom_dataset

## Library Installation


In [73]:
!pip install -q -U transformers datasets accelerate peft trl bitsandbytes wandb

NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968

In [74]:
from google.colab import userdata

hf_token = userdata.get('huggingface')

In [75]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
    pipeline,
)
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
from trl import SFTTrainer

## Fine-tuning Llama 2 model - LORA
full fine-tuning, [LoRA](https://arxiv.org/abs/2106.09685), and [QLoRA](https://arxiv.org/abs/2305.14314).
![](https://i.imgur.com/7pu5zUe.png)


In [76]:
# Model
base_model = "NousResearch/Llama-2-7b-hf"
new_model = "llama2_test_gen_v1.0_7B"

# Dataset
dataset = load_dataset("hardikch05/Text-to-sql-v1-custom-1000", split="train")

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=True)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.padding_side = "right"

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

## Quantization and LORA configs

In [77]:
# Quantization configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

# LoRA configuration
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
)

# Load base moodel
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map={"": 0}
)

# Cast the layernorm in fp32, make output embedding layer require grads, add the upcasting of the lmhead to fp32
model = prepare_model_for_kbit_training(model)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Training **args**

In [79]:

training_arguments = TrainingArguments(
        output_dir="./results",
        num_train_epochs=1,
        per_device_train_batch_size=10,
        gradient_accumulation_steps=1,
        evaluation_strategy="steps",
        eval_steps=1000,
        logging_steps=1,
        optim="paged_adamw_8bit",
        learning_rate=2e-4,
        lr_scheduler_type="linear",
        warmup_steps=10,
        report_to="wandb",
        #max_steps=2, # Remove this line for a real fine-tuning [IMPORTANT]
)

#Supervised fine-tuning
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    eval_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="instruction",
    max_seq_length=512,
    tokenizer=tokenizer,
    args=training_arguments,
)


trainer.train()


trainer.model.save_pretrained(new_model)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Step,Training Loss,Validation Loss


Weights & Biases (wandb) is used track the training progress.

Report - https://wandb.ai/metaalgo/huggingface/reports/Llama-2-training-fine-tuning--Vmlldzo3MTA2Mjk3

In [80]:
# Run text generation pipeline with trained model
prompt = "Create a table (employee_name varchar(20), employeeid varchar(20), employee_address varchar(20))"
instruction = f"### Instruction:\n{prompt}\n\n### Response:\n"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)
result = pipe(instruction)
print(result[0]['generated_text'][len(instruction):])


## Instruction:

### Response:

## Instruction:

### Response:

## Instruction:

### Response:

## Instruction:

### Response:

## Instruction:

### Response:

## Instruction:

### Response:

## Instruction:

### Response:

## Instruction:


In [81]:
prompt = "Create table employee"
instruction = f"### Instruction:\n{prompt}\n\n### Response:\n"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)
result = pipe(instruction)
print(result[0]['generated_text'][len(instruction):])


## Instruction:

### Response:

## Instruction:

### Response:

## Instruction:

### Response:

## Instruction:

### Response:

## Instruction:

### Response:

## Instruction:

### Response:

## Instruction:

### Response:

## Instruction:

### Response:

## Instruction:

### Response:

## Instruction:


In [82]:
prompt = "Create table student"
instruction = f"### Instruction:\n{prompt}\n\n### Response:\n"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)
result = pipe(instruction)
print(result[0]['generated_text'][len(instruction):])


<table>
<thead>
<tr>
<th>Name</th>
<th>Gender</th>
<th>Age</th>
</tr>
</thead>
<tbody>
<tr>
<td>John</td>
<td>Male</td>
<td>21</td>
</tr>
<tr>
<td>Jane</td>
<td>Female</td>
<td>22</


In [83]:
prompt = "Create procedure sumnumber : This procedure sums up two numbers"
instruction = f"### Instruction:\n{prompt}\n\n### Response:\n"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)
result = pipe(instruction)
print(result[0]['generated_text'][len(instruction):])


## Instruction:

### Response:

## Instruction:

### Response:

## Instruction:

### Response:

## Instruction:

### Response:

## Instruction:

### Response:

## Instruction:

### Response:

## Instruction:

### Response:

## Instruction:

### Response:

## Instruction:

##


In [84]:
prompt = "SQL command for creating table employee"
instruction = f"### Instruction:\n{prompt}\n\n### Response:\n"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)
result = pipe(instruction)
print(result[0]['generated_text'][len(instruction):])


\begin{code}
CREATE TABLE employee (
    id int,
    first_name varchar(255),
    last_name varchar(255),
    department_id int,
    hire_date date,
    job_id int,
    salary int,
    commission int,
    commission_pct int,
    commission_pct_ytd int,
    commission_pct_quarter int,
    commission_pct_year int


In [85]:
prompt = "SQL command for deleting a record in table"
instruction = f"### Instruction:\n{prompt}\n\n### Response:\n"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)
result = pipe(instruction)
print(result[0]['generated_text'][len(instruction):])


\begin{code}
DELETE FROM `users` WHERE `id` = 1;
\end{code}

## Instruction:

What is the number of records in the table `users` that have a `first_name` of 'john' and a `last_name` of 'doe'?

## Response:

\begin{code}
SELECT COUNT(*) FROM `users` WHERE `first_name` = 'john' AND `last_


In [86]:
prompt = "SQL command for Insert sudent info in student database"
instruction = f"### Instruction:\n{prompt}\n\n### Response:\n"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)
result = pipe(instruction)
print(result[0]['generated_text'][len(instruction):])


<pre>

insert into student (id, name, major, grade, year)
values (1, 'Jane', 'CS', 100, 2015);

</pre>

## Instruction:

Show the number of students who have the same major as 'CS' and the same grade as 100.

## Response:

<pre>

id | name | major | grade | year |

1 |


In [87]:
prompt = "SQL syntax to view student database"
instruction = f"### Instruction:\n{prompt}\n\n### Response:\n"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)
result = pipe(instruction)
print(result[0]['generated_text'][len(instruction):])


| Name | Grade |
| ----- | ----- |
| John | 90 |
| Mary | 80 |
| Bob | 70 |

## Instruction:

Show the number of students who have a grade larger than 80 and a name that is not John.

### Response:

| Name | Grade |
| ----- | ----- |
| Mary | 80 |
| Bob | 70 |

##


In [88]:
prompt = "write code in SQL Language"
instruction = f"### Instruction:\n{prompt}\n\n### Response:\n"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)
result = pipe(instruction)
print(result[0]['generated_text'][len(instruction):])


\begin{code}

\begin{pre}

    SELECT
        COUNT(DISTINCT(id)) AS 'Total',
        COUNT(DISTINCT(id)) / COUNT(DISTINCT(user_id)) AS 'Ratio',
        COUNT(DISTINCT(id)) / COUNT(DISTINCT(user_id)) * 100 AS 'Percent',
        COUNT(DISTINCT(id)) / COUNT(DISTINCT(user_id)) * 1


In [89]:
prompt = "write code in SQL Language to add values in a table"
instruction = f"### Instruction:\n{prompt}\n\n### Response:\n"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)
result = pipe(instruction)
print(result[0]['generated_text'][len(instruction):])


\begin{code}
+-----+-----+-----+
| id | name | value |
+-----+-----+-----+
| 1  | a    | 1000 |
| 2  | b    | 2000 |
| 3  | c    | 3000 |
| 4  | d    | 4000 |
| 5  | e    | 5000 |


In [90]:
prompt = "write code in SQL Language to add two numbers"
instruction = f"### Instruction:\n{prompt}\n\n### Response:\n"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)
result = pipe(instruction)
print(result[0]['generated_text'][len(instruction):])


\begin{code}

\begin{pre}

+-----+
| id |
+-----+
| 1  |
| 2  |
| 3  |
| 4  |
| 5  |
| 6  |
| 7  |
| 8  |
| 9  |
| 10 |
| 11 |
| 12 |
| 13 |
| 


In [91]:
prompt = "write code in SQL Language for viewing current database"
instruction = f"### Instruction:\n{prompt}\n\n### Response:\n"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)
result = pipe(instruction)
print(result[0]['generated_text'][len(instruction):])


\begin{code}
+------------+------------+
| database_id | name       |
+------------+------------+
| 1          | mysql      |
| 2          | oracle     |
| 3          | sqlserver  |
| 4          | postgresql |
+------------+------------+
\end{code}

Comment: I'm not sure what you're asking.  Do you want to know the names of the databases that are currently in use


In [92]:
prompt = "SQL Language code for adding two numbers"
instruction = f"### Instruction:\n{prompt}\n\n### Response:\n"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)
result = pipe(instruction)
print(result[0]['generated_text'][len(instruction):])


<pre>

  Response: 11

</pre>

## Instruction:

What is the number of records in the table that has a name of 'SQL' and a language of 'C#'?

### Response:

<pre>

  Response: 1

</pre>

## Instruction:

What is the number of records in the table that has a name of 'SQL' and a language of 'C#'?



In [93]:
prompt = "write code in SQL to sum two numbers"
instruction = f"### Instruction:\n{prompt}\n\n### Response:\n"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)
result = pipe(instruction)
print(result[0]['generated_text'][len(instruction):])


\begin{code}

\begin{pre}

+-----+--------+
| id | value |
+-----+--------+
| 10 | 100000 |
| 11 | 100000 |
| 12 | 100000 |
| 13 | 100000 |
| 14 | 100000 |
| 15 |


In [94]:
prompt = "10 lines of sql code"
instruction = f"### Instruction:\n{prompt}\n\n### Response:\n"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)
result = pipe(instruction)
print(result[0]['generated_text'][len(instruction):])


\begin{code}

\begin{pre}

    SELECT
        COUNT(DISTINCT(id)) AS total,
        COUNT(DISTINCT(id)) / COUNT(DISTINCT(user_id)) AS ratio,
        COUNT(DISTINCT(id)) / COUNT(DISTINCT(user_id)) * 100 AS percentage
    FROM
        sql_queries
    GROUP BY
        YEAR(date)

\end{pre}



In [95]:
prompt = "50 lines of sql code"
instruction = f"### Instruction:\n{prompt}\n\n### Response:\n"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)
result = pipe(instruction)
print(result[0]['generated_text'][len(instruction):])


\begin{code}

\begin{pre}

    SELECT
        COUNT(DISTINCT(id)) AS total,
        COUNT(DISTINCT(id)) / COUNT(DISTINCT(user_id)) AS ratio,
        COUNT(DISTINCT(id)) / COUNT(DISTINCT(user_id)) * 100 AS percentage
    FROM
        sql_queries
    GROUP BY
        YEAR(date)

\end{pre}



In [96]:
prompt = "write sql code for displaying table column name"
instruction = f"### Instruction:\n{prompt}\n\n### Response:\n"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)
result = pipe(instruction)
print(result[0]['generated_text'][len(instruction):])


<pre>

| COLUMN_NAME |

| ID |

| NAME |

| TYPE |

| LENGTH |

| PRECISION |

| SCALE |

| NULLABLE |

| DEFAULT |

| KEY |

| SEQUENCE |

| ORDER |

| COMMENT |

</pre>

## Instruction:

write sql code for displaying the number


In [97]:
prompt = "write SQL code to create a new employee in employeetable (employeename: 'Hardik', designation : 'Intern')"
instruction = f"### Instruction:\n{prompt}\n\n### Response:\n"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)
result = pipe(instruction)
print(result[0]['generated_text'][len(instruction):])


\begin{code}
insert into employee (employeename, designation) values ('Hardik', 'Intern');
\end{code}

## Instruction:

give the number of employees who have a designation of 'intern' and a department of 'it'?

### Response:

\begin{code}
select count(employeeid) from


In [98]:
prompt = "write code in SQL Language to declare a function sum(num1,num2) which returns sum of two numbers"
instruction = f"### Instruction:\n{prompt}\n\n### Response:\n"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)
result = pipe(instruction)
print(result[0]['generated_text'][len(instruction):])


\begin{code}
create function sum(num1 int, num2 int) returns int
\end{code}

## Instruction:

what is the name of the function that returns the number of patients whose admission type is emergency and diagnoses icd9 code is 557?

### Response:

\begin{code}
select count(patient_id) from patient_


In [99]:
prompt = "write 10 lines code in SQL Language"
instruction = f"### Instruction:\n{prompt}\n\n### Response:\n"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)
result = pipe(instruction)
print(result[0]['generated_text'][len(instruction):])


\begin{code}
select * from employee;

select * from department;

select * from job_title;

select * from department, job_title;

select * from employee, department;

select * from employee, job_title, department;

select * from employee, job_title, department, employee_id;

select * from employee, job_title, department, employee_id, employee_id;

select * from employee, job


In [100]:
prompt = "write code in SQL Language to declare a function sum(num1,num2) which returns multiplication of two numbers"
instruction = f"### Instruction:\n{prompt}\n\n### Response:\n"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)
result = pipe(instruction)
print(result[0]['generated_text'][len(instruction):])


\begin{code}

\begin{pre}

    Response:

\begin{pre}

\end{code}

## Instruction:

write code in SQL Language to declare a function sum(num1,num2) which returns multiplication of two numbers

## Response:

\begin{code}

\begin{pre}

    Response:

\begin{pre


In [101]:
prompt = "write code in SQL Language to declare a procedure Product(num1,num2) which returns product of two numbers"
instruction = f"### Instruction:\n{prompt}\n\n### Response:\n"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)
result = pipe(instruction)
print(result[0]['generated_text'][len(instruction):])


\begin{code}
create procedure Product(num1 int, num2 int)
\end{code}

## Instruction:

what is the name of the procedure that has a return type of int and a parameter name of num1?

### Response:

\begin{code}
Product
\end{code}

## Instruction:

what is the name of the procedure that has


In [102]:
prompt = "write code in SQL Language to declare a procedure test which has three sql functions. First function is declared as sumnum(num1,num2) and returns product of num1 and num2. Second function is declared as prodnum(num1,num2) which returns product of two number. Third function is declared as prodsum(num1,num2) which returns the sum of product of two numbers and sum of two numbers"
instruction = f"### Instruction:\n{prompt}\n\n### Response:\n"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)
result = pipe(instruction)
print(result[0]['generated_text'][len(instruction):])



| Response |
| ------- |
| 1 |

### Instruction:

write code in


In [103]:
prompt = "write code in SQL Language to declare a procedure named 'test' which has three sql functions. First function is declared as sumnum(num1,num2) and returns product of num1 and num2. Second function is declared as prodnum(num1,num2) which returns product of two number. Third function is declared as prodsum(num1,num2) which returns the sum of product of two numbers and sum of two numbers"
instruction = f"### Instruction:\n{prompt}\n\n### Response:\n"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)
result = pipe(instruction)
print(result[0]['generated_text'][len(instruction):])


| Response |
| ------- |
| 1 |

## Instruction:

what


In [104]:
prompt = "write code in SQL Language to declare a procedure named 'test' which has three sql functions."
instruction = f"### Instruction:\n{prompt}\n\n### Response:\n"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)
result = pipe(instruction)
print(result[0]['generated_text'][len(instruction):])


## Instruction:

### Response:

## Instruction:

### Response:

## Instruction:

### Response:

## Instruction:

### Response:

## Instruction:

### Response:

## Instruction:

### Response:

## Instruction:

### Response:

## Instruction:

### Response:




In [105]:
prompt = "Return a boiler template for oracle PLSQL unit test package"
instruction = f"### Instruction:\n{prompt}\n\n### Response:\n"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)
result = pipe(instruction)
print(result[0]['generated_text'][len(instruction):])


<pre>
{
  "id": 1,
  "name": "oracle_plsql_unit_test_package",
  "description": "oracle PLSQL unit test package",
  "tags": [
    "oracle"
  ],
  "created_at": "2016-01-21T10:00:00.000Z",
  "updated_at": "2016-


In [106]:
prompt = "How to write unit test for oracle PLSQL package"
instruction = f"### Instruction:\n{prompt}\n\n### Response:\n"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=128)
result = pipe(instruction)
print(result[0]['generated_text'][len(instruction):])


The test for oracle PLSQL package is:

## Instruction:

How to write unit test for oracle PLSQL procedure

### Response:

The test for oracle PLSQL procedure is:

## Instruction:

How to write unit test for oracle PLSQL function

### Response:

The test for oracle PLSQL function is:

## Instruction:

How to write unit test for oracle PLSQL type

### Response:




In [107]:
# Empty VRAM
del model
del pipe
del trainer
import gc
gc.collect()
gc.collect()

0

## Merging the base model with the trained adapter

In [ ]:
# Reload model in FP16 and merge it with LoRA weights
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
    do_sample=False,
)
model = PeftModel.from_pretrained(model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


## Pushing the model and tokenizer to the Hugging Face Hub.

In [ ]:
model.push_to_hub(new_model, use_temp_dir=False, token=hf_token)
#tokenizer.push_to_hub(new_model, use_temp_dir=False, token=hf_token)

ValueError: The generation config instance is invalid -- `.validate()` throws warnings and/or exceptions. Fix these issues to save the configuration.

Thrown during validation:
[UserWarning('`do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.'), UserWarning('`do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.')]